#MOUNT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#IMPORTS

In [ ]:
import spacy
from spacy import displacy
from spacy import tokenizer
import tweepy
import datetime
import sys
import os
import csv
from itertools import combinations

#TWITTER AUTHENTICATION

In [ ]:
client = tweepy.Client("AAAAAAAAAAAAAAAAAAAAAJFkOwEAAAAANz6uxV4%2BbHJOMd6B8waFjGD71No%3D76cdhvol60LR2s1T4bakR0KTqV2f6V4Cfa2FZ1u01rad8pIMOd",wait_on_rate_limit=True)
auth = tweepy.OAuth2BearerHandler("AAAAAAAAAAAAAAAAAAAAAJFkOwEAAAAANz6uxV4%2BbHJOMd6B8waFjGD71No%3D76cdhvol60LR2s1T4bakR0KTqV2f6V4Cfa2FZ1u01rad8pIMOd")
api = tweepy.API(auth,wait_on_rate_limit=True)
consumer_key = "86xM43yUqYsDOVEEG9l4QSlTE"
consumer_secret = "W1jB00fdVVzRzyQd8p9mJnzKC9XAGQRqCP5G4hRKnoA3KM1Dna"
access_token = "1156738416-CiGsQPHyWVUieqnCyDfWBz3OLqfpKDpiljj9zcr"
access_token_secret = "P8UDrYvC9LnOIblV5WWgBvoTXkjNqHxzN2PUTyHivC0EW"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
nlp = spacy.load('en_core_web_sm')

counter = 0
total = 0

#CODE

In [ ]:
with open('label.txt') as topo_file:
    for line in topo_file:
        tweet_id = ""
        tweet_text = ""

        if(line[0]=='f'):
          tweet_id = line[6:24]
          tweet_id = str(tweet_id)
          cnt = 0
          for x in tweet_id:
            if(x.isnumeric()):
              cnt += 1
          if(cnt == 18):
            tweet_text = client.get_tweet(tweet_id).data

        elif(line[0]=='t'):
          tweet_id = line[5:23]
          tweet_id = str(tweet_id)
          cnt = 0
          for x in tweet_id:
            if(x.isnumeric()):
              cnt += 1
          if(cnt == 18):
            tweet_text = client.get_tweet(tweet_id).data

        else:
          continue

        print(tweet_id,tweet_text)

        tweet_date = ''

        if(tweet_text != None):
            total += 1
            tweet_date = api.get_status(tweet_id).created_at
            tweet_hashtags = api.get_status(tweet_id).entities['hashtags']
            counter += len(tweet_hashtags)
            keywords = nlp(str(tweet_text))
            ents = [(e.text,e.label_) for e in keywords.ents]
            print(tweet_text)
            entities = []
            entities = set(entities)
            for x in tweet_hashtags:
                entities.add(x['text'])

            for x in ents:
                if(x[1]=='PERSON' or x[1]=='NORP' or x[1]=='ORG' or x[1]=='FAC' or x[1]=='GPE' or x[1]=='LOC' or x[1]=='EVENT' or x[1]=='PRODUCT'):
                    newx = ""
                    for letter in x[0]:
                        if(letter.isalnum() or letter == ' '):
                            newx += letter
                    entities.add(newx)

            entities = list(entities)
            tweet_crowd = []
            tweet_crowd = set(tweet_crowd)

            for r in range(0,len(entities)):
                realr = len(entities)-r
                arr = list(combinations(entities, realr))
                done = 0
                for dict in arr:
                    query = ""
                    for ele in dict:
                        query += ele
                        query += " "
                    if(len(query)>0):
                        query = query [:len(query)-1]
                        start_time = tweet_date
                        end_time = tweet_date
                        start_time -= datetime.timedelta(days=1)
                        end_time += datetime.timedelta(days=1)
                        tweets = client.search_all_tweets(query=query,start_time=start_time,end_time=end_time, max_results=100)
                        tweets = tweets.data
                        if( tweets is not None):
                            for result in tweets:
                                tweet_crowd.add((result.text))
                                if(len(tweet_crowd)>=100):
                                    done =1
                                    break
                        if(len(tweet_crowd)>=100):
                            done = 1
                            break
                    if(done == 1):
                        break
                if(done == 1):
                    break

            if(line[0]=='t'):
              os.chdir('BTPDataset/true')
            else:
              os.chdir('BTPDataset/false')

            filename = tweet_id+'.csv'
            with open(filename, 'w') as csvfile:
                writer = csv.writer(csvfile)
                temprow = []
                temprow.append(tweet_text)
                writer.writerow(temprow)
                for x in tweet_crowd:
                    temprow = []
                    temprow.append(x)
                    writer.writerow(temprow)

            os.chdir('..')
            os.chdir('..')
